In [ ]:
# settings for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# turn off matplotlib figure axes, else we need to call axes('off') everytime
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.set_cmap('gray') 
# mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
# mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=20)

import warnings
warnings.filterwarnings("ignore")

# import necessarry libs
from imageio import imread
import numpy as np
from scipy.signal import convolve2d as conv2
from skimage.color import rgb2gray
from skimage.transform import resize
from scipy.fftpack import fft2, fftshift, ifft2, ifftshift
from roi import new_ROI

In [ ]:
# Lire l'image    
im = imread('img2.jpg')/255.0;
im = rgb2gray(im);# FFT
im_fft = fft2(im); 

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(10,5), num='Image')

ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(im); ax1.axis('off'); ax1.set_title('image')

ax2 = fig.add_subplot(1, 2, 2, sharex = ax1, sharey = ax1)
fmap = ax2.imshow(np.log(np.abs(fftshift(im_fft))));
ax2.set_title('im FFT'); ax2.axis('off')
fig.colorbar(fmap, ax=ax2,fraction=0.046, pad=0.04)
plt.tight_layout()

In [ ]:
# Re-construction
imRec = ifft2(im_fft).real;
fig = plt.figure(figsize=(15,5), num='Reconstruite')

ax1 = fig.add_subplot(1, 3, 1)
fmap = ax1.imshow(im); ax1.axis('off'); ax1.set_title('Image originale')

ax2 = fig.add_subplot(1, 3, 2, sharex = ax1, sharey = ax1)
fmap = ax2.imshow(imRec); ax2.axis('off'); ax2.set_title('Image reconstruite')

ax3 = fig.add_subplot(1, 3, 3, sharex = ax1, sharey = ax1)
fmap = ax3.imshow(np.abs(im-imRec)*100); ax3.axis('off'); ax3.set_title('Erreur (x100)') 
fig.colorbar(fmap, ax=ax3,fraction=0.046, pad=0.04)
plt.tight_layout()

In [ ]:
# Masquons une partie des fréquences, et re-construisons...
im_fft_shift = fftshift(im_fft);
plt.figure(figsize=(10,5), num='Draw ROI')
# log(1+x) to avoid negative inf, this is a bit different visualization than the matlab code
im_axis = plt.imshow(np.log(1+np.abs(im_fft_shift)));plt.axis('off')
roi = new_ROI(im_axis, shape='polygon')

In [ ]:
masque = roi.get_mask()
assert np.any(masque[:]) == True, 'roi is empty'

In [ ]:
# masquons les fréquences correspondantes
im_fft_shift[masque] = 0
imRecMasquee = np.abs(ifft2(ifftshift(im_fft_shift)).real);

fig, ax = plt.subplots(1,3, figsize=(15,5), sharex=True, sharey=True,num='reconstruite')

ax[0].imshow(im); ax[0].axis('off'); ax[0].set_title('Image originale')
ax[1].imshow(imRecMasquee); ax[1].axis('off'); ax[1].set_title('Image reconstruite');
# log(1+x) to avoid negative inf
im_fft_shift_log = np.log(1+np.abs(im_fft_shift))
fmap = ax[2].imshow(im_fft_shift_log); ax[2].set_title('fft_shift'); ax[2].axis('off');
fig.colorbar(fmap, ax=ax[2],fraction=0.046, pad=0.04);